### Spark 의 구조와 매커니즘

시작하기 전 스파크의 내부 설계도와 잊지말아야 할 점.


1 - 내부설계도

<img src = 'https://drive.google.com/uc?id=1Yh50tMjNsjiMRwNXDMmx_7etPSG4FRiz'>

2 - 항상 생각할 점 

__언제나 데이터가 여러 곳에 분산돼 저장된다는 점__

__같은 연산도 여러 노드에 걸쳐서 실행된다는 점__

설계도를 보면 알 수 있는 것처럼 스파크에는 Driver(master) 라고 부르는 프로그램과 Worker 라고 부르는 노드들이 있다.

드라이버 프로그램(master node)에는 모든 프로세스를 조직하고 메인 프로세스를 수행한다.

그리고 SparkContext 가 있고 SparkContext 는 dataframe, list, array 를 RDD 로 만들 수 있다.

그 method로는 .textFile(), .parallelize() 가 있고 보통 textfile() 의 경우 큰 볼륨의 파일을 다룰 때 사용하고,

parallelize()의 경우엔 작은 로컬데이터를 사용할 때 이용한다. 

RDD로 만든 후에는 Transformation 으로 변환을 하고 Action method 를 사용해서 python object 로 만들거나 수정할 수 있다. (spark repository 에서 확인 가능)

보통 driver 의 경우에는 개발 단계에서 user가 프로그램과 상호작용을 하는 프로그램이고 실제 작업들은 worker node 에서 일어난다.

dirver 와 worker 의 통신은 cluster manager 가 담당한다. 

cluster manager 는 task들에 대한 scheduling, resource managing 을 한다. 

보통 yarn, mesos 를 cluster manager 로 사용한다.

Worker node 에서는 __'내부의 executer 가'__ 연산(master node 가 전달한 task들) or 데이터를 처리 or master node 로 정보 전달(heartbeat 라고 함. '나 살아있어요' 상태보고)

전에 말했던 메소드 중에서 .foreach() 라는 메소드를 기억해주면 좋은데, 그 때 'RDD.foreach() 메소드를 사용해도 driver 에서는 확인할 수 없지만 Worker node 에서는 실행되고 있다.'

라고 말한 적이 있는데, 그 worker node 가 지금 말하는 worker node 이다. 코드가 아마 RDD.foreach(lambda x:print(x)) 였던 거 같은데?? __action 설명파트 참조__

그리고 개인적으로 중요하다고 생각하는 점은 cache 부분이다. 저 cache 부분이 중요한 이유는 자주 사용되는 중간 결과나 계산된 데이터를 메모리에 저장해서 

반복적인 계산이 필요할 때 매번 데이터를 다시 계산하는 것을 방지해주는 역할임. 

spark 는 master node 가 여러 worker node 들로 task 를 분산해서 작업을 진행한다는 점을 먼저 remind 드리며,  

그리고 cache 를 진행할 때 spark 는 특정한 RDD 단위로 data caching 을 하기 때문에 여러 노드들에서 작업이 진행돼도 데이터가 중복으로는 caching 이 되지않는다는 점 !

그래서 최종적으로 그렇게 처리된 데이터들을  action method 를 사용해서 driver 로 데이터를 하나의 파일처럼 받아볼 수 있다는 것 ! 

RDD.take(3) 이런 거? collect() 사용을 지양하자는 말도 했었음. __action 설명파트 참조__

참 재미있는 부분이 데이터를 저장할 당시엔 여러 노드에 분산시켜 저장했지만, 사용할 때는 하나의 파일처럼 사용할 수 있다라는 점 ! 

마무리 정리를 하면 'Spark' 는 

driver 에서 요청된 task 들은 cluster manager 를 통해 worker node 들에서 분산돼 연산하고, 

그렇게 "data caching 을 이용해 저장된 분산된 데이터들은 사용할 때는 다시 하나의 것처럼 사용한다" 가 핵심이다.